递归神经网络被广泛的应用于语言与时序分析领域，在这篇博客中我们将会用简单的例子演示递归神经网络的使用

第一步让我们读取ptb数据

In [1]:
import collections
import os

import tensorflow as tf


def _read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()


def _build_vocab(filename):
    data = _read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def _file_to_word_ids(filename, word_to_id):
    data = _read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]


def ptb_raw_data(data_path=None):
    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    word_to_id = _build_vocab(train_path)
    train_data = _file_to_word_ids(train_path, word_to_id)
    valid_data = _file_to_word_ids(valid_path, word_to_id)
    test_data = _file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    return train_data, valid_data, test_data, vocabulary


def ptb_producer(raw_data, batch_size, num_steps, name=None):
    with tf.name_scope(name, "PTBProducer", [raw_data, batch_size, num_steps]):
        raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)

        data_len = tf.size(raw_data)
        batch_len = data_len // batch_size
        data = tf.reshape(raw_data[0 : batch_size * batch_len],
                          [batch_size, batch_len])

        epoch_size = (batch_len - 1) // num_steps
        assertion = tf.assert_positive(
            epoch_size,
            message="epoch_size == 0, decrease batch_size or num_steps")
        with tf.control_dependencies([assertion]):
              epoch_size = tf.identity(epoch_size, name="epoch_size")

        i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
        x = tf.strided_slice(data, [0, i * num_steps],
                             [batch_size, (i + 1) * num_steps])
        x.set_shape([batch_size, num_steps])
        y = tf.strided_slice(data, [0, i * num_steps + 1],
                             [batch_size, (i + 1) * num_steps + 1])
        y.set_shape([batch_size, num_steps])
        return x, y


In [2]:
batch_size = 200
num_steps = 10

train_data, valid_data, test_data, vocabulary = ptb_raw_data("data/ptb/")
#x, y = ptb_producer(valid_data, batch_size, num_steps)

raw_data = valid_data
with tf.name_scope("train", "PTBProducer", [raw_data, batch_size, num_steps]):
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)

    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0 : batch_size * batch_len],
                      [batch_size, batch_len])

    epoch_size = (batch_len - 1) // num_steps
    assertion = tf.assert_positive(
        epoch_size,
        message="epoch_size == 0, decrease batch_size or num_steps")
    with tf.control_dependencies([assertion]):
          epoch_size = tf.identity(epoch_size, name="epoch_size")

    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = tf.strided_slice(data, [0, i * num_steps],
                         [batch_size, (i + 1) * num_steps])
    x.set_shape([batch_size, num_steps])
    y = tf.strided_slice(data, [0, i * num_steps + 1],
                         [batch_size, (i + 1) * num_steps + 1])
    y.set_shape([batch_size, num_steps])


In [3]:
hidden_size = 100
is_training = True

embedding = tf.get_variable("embedding", [vocabulary, hidden_size], dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, x)

if is_training:
      inputs = tf.nn.dropout(inputs, 0.5)

In [4]:
cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True, reuse=not is_training)
if is_training:
      cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.5)

_initial_state = cell.zero_state(batch_size, tf.float32)
state = _initial_state

outputs = []
with tf.variable_scope("RNN"):
    for time_step in range(num_steps):
        if time_step > 0: 
            tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)
output = tf.reshape(tf.concat(outputs, 1), [-1, hidden_size])


In [5]:
softmax_w = tf.get_variable(
    "softmax_w", [hidden_size, vocabulary], dtype=tf.float32)
softmax_b = tf.get_variable("softmax_b", [vocabulary], dtype=tf.float32)
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
 # Reshape logits to be a 3-D tensor for sequence loss
logits = tf.reshape(logits, [batch_size, num_steps, vocabulary])

# Use the contrib sequence loss and average over the batches
loss = tf.contrib.seq2seq.sequence_loss(
    logits,
    y,
    tf.ones([batch_size, num_steps], dtype=tf.float32),
    average_across_timesteps=False,
    average_across_batch=True)


In [6]:
_cost = tf.reduce_sum(loss)
_final_state = state

_lr = tf.Variable(0.0, trainable=False)
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(_cost, tvars), 5)
optimizer = tf.train.GradientDescentOptimizer(_lr)
_train_op = optimizer.apply_gradients(
    zip(grads, tvars),
    global_step=tf.contrib.framework.get_or_create_global_step())

_new_lr = tf.placeholder(
    tf.float32, shape=[], name="new_learning_rate")
_lr_update = tf.assign(_lr, _new_lr)

In [7]:
init_scale = 0.04
initializer = tf.random_uniform_initializer(-init_scale, init_scale)

ops = {
    "cost": _cost,
    "lr": _lr,
    "new_lr": _new_lr,
    "lr_update": _lr_update,
}

for name, op in ops.items():
    tf.add_to_collection(name, op)
    
tf.add_to_collection("initial", _initial_state.c)
tf.add_to_collection("initial", _initial_state.h)
tf.add_to_collection("final", _final_state.c)
tf.add_to_collection("final", _final_state.h)


In [10]:
import time


max_max_epoch = 1
max_epoch = 1
_lr_decay = 0.8
learning_rate = 1.

sess = tf.InteractiveSession()

tupled_initial_state = tuple([tf.contrib.rnn.LSTMStateTuple(_initial_state.c, _initial_state.h)])

sess.run(tf.global_variables_initializer())


In [ ]:
for i in range(max_max_epoch):
    lr_decay = _lr_decay ** max(i + 1 - max_epoch, 0.0)
    sess.run(_lr_update, feed_dict={_new_lr: learning_rate * lr_decay})
    
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = sess.run(tupled_initial_state)

    fetches = {
        "cost": _cost,
        "final_state": _final_state,
    }

    for step in range(((len(valid_data) // batch_size) - 1) // num_steps):
        feed_dict = {}
        for i, (c, h) in enumerate(tupled_initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h

        vals = sess.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]

        costs += cost
        iters += num_steps

        print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / epoch_size, np.exp(costs / iters),
             iters * batch_size * max(1, FLAGS.num_gpus) /
             (time.time() - start_time)))


In [19]:
sess = tf.InteractiveSession()

sess.run(data)

array([[1132,   93,  358, ...,   69,  123,   10],
       [  14,   24,    7, ..., 6208,    5,  462],
       [  52,  104,  604, ..., 6485,    8, 4113],
       ..., 
       [2894,    4,  184, ...,   47, 3623,   46],
       [7851,    6,  470, ...,  132,    4, 9976],
       [   9,  346, 1108, ...,   12,    3,   21]], dtype=int32)

In [38]:
# 重置默认计算图
tf.reset_default_graph()
# 创建交互式计算会话
sess = tf.InteractiveSession()

train_data, valid_data, test_data, vocabulary = ptb_raw_data('data/ptb/')

sess.close()

data/ptb/ptb.train.txt
data/ptb/ptb.train.txt
data/ptb/ptb.valid.txt
data/ptb/ptb.test.txt
